Tests: 
Mapping changes to x wrt average distnace transforms 
Each test for changing x will have 2 graphs:
1. avg distance transform from pillar to catalyst layer
2. avg distance transfrom from oxygen surface to catalyst layer

Different test ideas:
I. Cylindrical Pillar
1. Changing radius constant
2. Changing height
3. Changing sloped radius (bottom constant, top changing)
4. Changing sloped radius (bottom changing, top constant)

II. Rectangular Pillar
1. Changing L:W Aspect Ratio 
2. Changing height
3. Changing sloped aspect ratio (bottom constant, top changing)
4. Changing sloped aspect ratio (bottom changing, top constant)

Requirements:
1. Function to generate a matrix and pillar structure for cylindrical
2. Same for rectangular

3. Avg distance transform helper function 

4. Plotting

Some additional readings to do:
1. Look into RL, Look into Bayesian

Cylinder Class:

1. Resolution (Sizing) must be high so that the curvature is approximated better 
2. Step size must be a constant integer so that each layer of the matrix has a constant integer increase/decrease

Implementation:

1. Enforce a x1000 scale-up on the original 25x25xh element design

2. Static factory (its not enforced in python sadly but whatever) to make checks on the arguments. This ensures that stepsize is both constant and integer, as the args directly impact step size.


In [86]:
import numpy as np
from scipy.interpolate import RegularGridInterpolator
from fractions import Fraction

class Cylinder():

    def __init__(self, r1, r2, h, hspace):
        self.r1 = r1
        self.r2 = r2
        self.h = h
        self.hspace = hspace

    @classmethod
    def create_cylinder(cls, r1, r2, h, hspace):
        slope = Fraction(r1-r2, h)
        if (slope % 1 != 0):
            raise ValueError("Step size is not an integer. This wont work for gradual slope increase for matrix")
        if (r2 > 25000):
            raise ValueError("Radii greater than element dimensions")
        return cls(r1, r2, h, hspace)

    def construct_space(self):
        # Create an empty matrix
        matrix_size = (25 * 1000, 25 * 1000, (self.h + self.hspace) * 1000)
        proton_matrix = np.zeros(matrix_size)

        # Calculate coordinates
        x = np.linspace(-25*1000, 25*1000-1, 25*1000*2)
        y = np.linspace(-25*1000, 25*1000-1, 25*1000*2)
        z = np.linspace(0, (self.h + self.hspace) * 1000 - 1, (self.h + self.hspace) * 1000)

        X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
        
        # Calculate radius at each height level
        radius = self.r1 - (self.r1 - self.r2) * (Z / (self.h * 1000))
        
        # Create a distance transform from the center
        dist_from_center = np.sqrt(X**2 + Y**2)
        
        # Fill the matrix where within the radius
        proton_matrix[dist_from_center <= radius] = 1

        return proton_matrix


For tomorrow: work on 

In [87]:
from fractions import Fraction

slope = Fraction(1-0, 100)
scale_factor = slope.denominator
print(slope)
print(scale_factor)

1/100
100


In [88]:
c1 = Cylinder.create_cylinder(r1=10, r2=5, h=5, hspace=0)

In [89]:
class Cylinder():

    def __init__(self, dims, r1, r2, h, hspace):
        self.dims = dims
        self.r1 = r1
        self.r2 = r2
        self.h = h
        self.hspace = hspace

    @classmethod
    def create_cylinder(cls, dims, r1, r2, h, hspace):
        slope = Fraction(r1-r2, h)
        if (slope % 1 != 0):
            raise ValueError(f"Step size is not an integer. Slope {slope}. This wont work for gradual slope increase for matrix")
        if (r2 > dims or r1 > dims):
            raise ValueError(f"Radii greater than element dimensions")
        return cls(dims, r1, r2, h, hspace)
    
    def construct_space(self):
        space = np.zeros((self.dims, self.dims, self.h+self.hspace))
        for layer in range(self.h):
            
        return space
    

c1 = Cylinder.create_cylinder(dims=5, r1=5, r2=1, h=4, hspace=7)
space = c1.construct_space()
print(space)

IndentationError: expected an indented block after 'for' statement on line 21 (68381895.py, line 23)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


space = np.zeros((10, 10, 10))
angles = np.linspace(0, 2*np.pi, 100)

r = 10
x = r*np.cos(angles)
y = r*np.sin(angles)




In [173]:
class Rect():
    def __init__(self, dims, x, y, origin):
        self.dims = dims
        self.x = x
        self.y = y
        self.origin = origin
    
    @classmethod
    def create_rect(cls, dims, x, y, origin: tuple):
        if (origin[0]+x > dims or origin[1]+y > dims):
            raise ValueError(f"size of rect greater than dimensions provided. origin = {origin} x = {x}, y = {y}, dims={dims}")
        return Rect(dims, x, y, origin)
    
    def make_layer(self):
        x = np.arange(self.dims)
        y = np.arange(self.dims)
        X, Y = np.meshgrid(x, y)

        left = self.origin[0]
        right = self.origin[0] + self.x - 1
        top = self.origin[1]
        bottom = self.origin[1] + self.y - 1

        mask = (X >= left) & (X <= right) & (Y >= top) & (Y <= bottom)

        return mask.astype(int)

In [174]:
r1 = Rect.create_rect(dims=5, x=2, y=2, origin=(0,0))
print(r1.make_layer())

# r2 = Rect.create_rect(dims=5, x=6, y=2, origin=(0,0))
# print(r2.make_layer())

# r3 = Rect.create_rect(dims=5, x=3, y=2, origin=(4,0))

[[1 1 0 0 0]
 [1 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [161]:
import numpy as np

class Circle():
    def __init__(self, dims, rx, ry, origin):
        self.dims = dims
        self.rx = rx
        self.ry = ry
        self.origin = origin
    
    @classmethod
    def create_circle(cls, dims, rx, ry, origin: tuple):
        if (origin[0] + rx + 1 > dims or origin[1] + ry + 1> dims) or (origin[0] - rx < 0 or origin[1] - ry < 0):
            raise ValueError(f"Circle does not fit in bounds defined by dims. Origin = {origin}, rx = {rx}, ry = {ry}, dims = {dims}")
        return Circle(dims, rx, ry, origin)
    
    def make_layer(self):
        x = np.arange(self.dims)
        y = np.arange(self.dims)
        X, Y = np.meshgrid(x, y)
        
        center_x = self.dims // 2
        center_y = self.dims // 2
        
        # Ellipse mask
        ellipse = ((X - center_x) ** 2 / self.rx ** 2 + (Y - center_y) ** 2 / self.ry ** 2) <= 1
        
        # Convert boolean array to integers
        return ellipse.astype(int)

In [167]:
c1 = Circle.create_circle(dims=10, rx=2, ry=4, origin=(5,5))
layer = c1.make_layer()
print(layer)
# c2 = Circle.create_circle(dims=100, x=20, y=20, origin=(0,50))
# c3 = Circle.create_circle(dims=100, x=70, y=20, origin=(50,50))

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 0 0 0]
 [0 0 0 1 1 1 1 1 0 0]
 [0 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]]


In [181]:
class Cylinder():
    def __init__(self, dims, height, r1x, r1y, r2x, r2y, origin):
        self.dims = dims
        self.height = height
        self.r1x = r1x
        self.r1y = r1y
        self.r2x = r2x
        self.r2y = r2y
        self.origin = origin
    
    @classmethod
    def create_cylinder(cls, dims, height, r1x, r1y, r2x, r2y, origin: tuple):
        if (origin[0] + r1x + 1 > dims or origin[1] + r1y + 1> dims) or (origin[0] - r1x < 0 or origin[1] - r1y < 0) or (origin[0] + r2x + 1 > dims or origin[1] + r2y + 1> dims) or (origin[0] - r2x < 0 or origin[1] - r2y < 0):
            raise ValueError(f"Circle does not fit in bounds defined by dims. Origin = {origin}, r1x = {r1x}, r1y = {r1y}, r2x = {r2x}, r2y = {r2y}, dims = {dims}")
        return Cylinder(dims, height, r1x, r1y, r2x, r2y, origin)
    
    def make_geometry(self):
        space = np.zeros((self.height, self.dims, self.dims), dtype=int)
        for i in range(self.height):
            circle = Circle.create_circle(dims=self.dims, rx=self.r1x, ry=self.r1y, origin=self.origin)
            space[i] = circle.make_layer()
        return space

In [190]:
c1 = Cylinder.create_cylinder(dims=250, height=5, r1x=60, r2x=60, r1y=30, r2y=30, origin=(120,120))
print(c1.make_geometry())

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
